# Regresión Lineal con Gradiente Descendente en el Boston Housing Dataset

**Nombre**: Edison Forero, Damian naranjo 
**Materia**: Ciencia de Datos  
**Salón**: TS7A

En este informe, se implementa una regresión lineal simple utilizando el **algoritmo de gradiente descendente**. El dataset utilizado es el **Boston Housing**, y la variable independiente seleccionada es el número de habitaciones por vivienda (`RM`), mientras que el objetivo es predecir el valor medio de las viviendas (`MEDV`).


In [3]:
import random
import numpy as np

# Número de ciudades
num_cities = 10

# Generar una matriz de distancias simétricas aleatorias entre ciudades
def generate_distance_matrix(num_cities):
    matrix = np.random.randint(1, 100, size=(num_cities, num_cities))
    # Hacer la matriz simétrica (distancia de A a B es igual que de B a A)
    matrix = (matrix + matrix.T) / 2
    # Poner ceros en la diagonal (distancia de una ciudad consigo misma es 0)
    np.fill_diagonal(matrix, 0)
    return matrix

distance_matrix = generate_distance_matrix(num_cities)
print("Matriz de distancias (dataset simulado):")
print(distance_matrix)


Matriz de distancias (dataset simulado):
[[ 0.  87.  55.  73.  58.5 48.  54.  59.  41.5 49.5]
 [87.   0.  48.  69.5 47.   7.5 45.  49.  36.  55.5]
 [55.  48.   0.  16.  60.5 29.5 65.5 78.5 51.5 76. ]
 [73.  69.5 16.   0.  36.5 66.5 14.5 67.5 24.  39.5]
 [58.5 47.  60.5 36.5  0.  47.  67.  17.5 94.  35.5]
 [48.   7.5 29.5 66.5 47.   0.  31.5 75.5 40.  43. ]
 [54.  45.  65.5 14.5 67.  31.5  0.  87.  30.  52. ]
 [59.  49.  78.5 67.5 17.5 75.5 87.   0.  32.5  2.5]
 [41.5 36.  51.5 24.  94.  40.  30.  32.5  0.  36.5]
 [49.5 55.5 76.  39.5 35.5 43.  52.   2.5 36.5  0. ]]


## 2. Definición del Problema y Funciones para el Algoritmo Genético

En esta parte se define el problema de optimización como una minimización, ya que queremos minimizar la distancia total del recorrido. Luego, se define la función para crear un individuo (una posible solución) y la función de evaluación que calcula la distancia total de un recorrido dado.


In [4]:
from deap import base, creator, tools, algorithms

# Crea el tipo de problema. Usamos minimización (-1.0) ya que queremos minimizar la distancia total.
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Función para crear un individuo (solución inicial)
def create_individual(city_list):
    return random.sample(city_list, len(city_list))

# Función de evaluación: Calcula la distancia total de un recorrido
def eval_tsp(individual, distance_matrix):
    total_distance = 0
    for i in range(len(individual) - 1):
        total_distance += distance_matrix[individual[i]][individual[i+1]]
    # Regresar al punto de partida
    total_distance += distance_matrix[individual[-1]][individual[0]]
    return total_distance,


## 3. Configuración del Toolbox del Algoritmo Genético

En esta sección configuramos el toolbox de DEAP. Esto incluye registrar los operadores genéticos como el cruce, la mutación y la selección. También se registra la función de evaluación que calcula la distancia de un recorrido.


In [ ]:
# 3. Configurar el toolbox del algoritmo genético
toolbox = base.Toolbox()

# Definir el conjunto de ciudades (lista de índices de ciudades)
city_list = list(range(len(distance_matrix)))

# Registra las funciones en el toolbox
toolbox.register("individual", tools.initIterate, creator.Individual, lambda: create_individual(city_list))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Operadores genéticos
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", eval_tsp, distance_matrix=distance_matrix)


## 4. Ejecución del Algoritmo Genético

En este paso, ejecutamos el algoritmo genético con una población inicial y un número definido de generaciones. Utilizamos una probabilidad de cruce y de mutación para evolucionar la población y encontrar la mejor solución.


In [ ]:
# Parámetros del algoritmo
population_size = 300
num_generations = 400
prob_crossover = 0.7  # Probabilidad de cruce
prob_mutation = 0.2   # Probabilidad de mutación

# Crear la población inicial
population = toolbox.population(n=population_size)

# Ejecutar el algoritmo genético
best_solutions = algorithms.eaSimple(population, toolbox, cxpb=prob_crossover, mutpb=prob_mutation, 
                                     ngen=num_generations, verbose=True)


## 5. Resultados

Finalmente, mostramos los resultados obtenidos, como el mejor recorrido encontrado y la distancia total de ese recorrido.


In [ ]:
# 5. Mostrar los resultados

# Obtener el mejor individuo de la población
best_individual = tools.selBest(population, k=1)[0]

# Imprimir el mejor recorrido y la distancia total
print(f"Mejor recorrido encontrado: {best_individual}")
print(f"Distancia total: {eval_tsp(best_individual, distance_matrix)[0]}")
